<p> import necessary libraries </p>

In [1]:
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
from scipy.stats import mode
import operator
import functools 
import statistics

<p> read the csv </p>

In [2]:
df_2012 = pd.read_csv('BPI_Challenge_2012-training.csv')

In [3]:
#df_2012

<p> Assign position numbers to events. Also include the case number for a table that gives only the position number of events, the case number and the event name. </p>

In [4]:
def df_filter_cols(df_original):
    df_cols1 = df_original.set_index(df_original.groupby('case concept:name').cumcount(), append = True)
    df_cols1 = df_cols1.reset_index()
    df_cols2 = df_cols1[["level_1", "case concept:name", "event concept:name", "event lifecycle:transition"]]
    return df_cols2

df_cols = df_filter_cols(df_2012)
df_cols


,level_1,case concept:name,event concept:name,event lifecycle:transition
0,0,173688,A_SUBMITTED,COMPLETE
1,1,173688,A_PARTLYSUBMITTED,COMPLETE
2,2,173688,A_PREACCEPTED,COMPLETE
3,3,173688,W_Completeren aanvraag,SCHEDULE
4,0,173691,A_SUBMITTED,COMPLETE
5,1,173691,A_PARTLYSUBMITTED,COMPLETE
6,2,173691,A_PREACCEPTED,COMPLETE
7,3,173691,W_Completeren aanvraag,SCHEDULE
8,0,173694,A_SUBMITTED,COMPLETE
9,1,173694,A_PARTLYSUBMITTED,COMPLETE


<p> Basic idea: Make a dictionary, that has (1) the case number (as key) and (2) an array belonging to this case number with in order, all of the events of this trace. This 2d array can then be used to calculate the distance between a given trace and all of the other traces in the log. </p>

In [5]:
case_events = {"case_1":["A", "B", "C"], "case_2":["B", "C", "C", "A"], "case_3":["D", "C", "B", "A"], "case_4":["A", "C"]}
#print(case_events["case_1"])


<p> The algorithm that calculates the distance between two arrays. For now I've used two placeholder arrays to test the algorithm. Below I am working on code to get the input from the df for this algorithm. </p>

In [6]:
#calculate the distance between arrays a and b
a = [0, "a", "b", "c", "d"]
b = [0, "a", "c", "b", "d"]
d = [[]]
#need to make d of specified length beforehand?? Probably just need a different data type because this is garbageeeeeeeee

for i in range (0, len(a)):
    d.append([i])

for j in range (0, len(b)):
    d[0].insert(j, j)
    
print(d)

for i in range (1, len(a)):
    for j in range (1, len(b)):
        if a[i] == b[j]:
            cost = 0
        else:
            cost = 1
        d[i].insert(j, min(d[i-1][j] + 1,
                     d[i][j-1] + 1,
                     d[i-1][j-1]+cost))
        if i>1 and j>1 and a[i] == b[j-1] and a[i-1] == b[j]:
            d[i].insert(j, min(d[i][j], d[i-2][j-2] + 1))
            d[i].pop(j+1)
print("given array a and b, the distance between array a and b is: ")
print(d[len(a)-1][len(b)-1])


[[0, 1, 2, 3, 4], [0], [1], [2], [3], [4]]
given array a and b, the distance between array a and b is: 
1


<p> Create a dictionary with arrays of events for all of the cases in the df. </p>

In [7]:
def create_case_dictionary(df):
    df["case concept:name"].unique()
    case_events={}
    df.sort_values("level_1")

    for index, row in df.iterrows():
        if row["level_1"] == 0:
            case_events.update({row["case concept:name"] : ["X"]})
        case_events.get(row["case concept:name"]).append(row["event concept:name"])
    return case_events

case_events = create_case_dictionary(df_cols)
print(case_events.get(173688))

['X', 'A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED', 'W_Completeren aanvraag', 'W_Completeren aanvraag', 'A_ACCEPTED', 'A_FINALIZED', 'O_SELECTED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes', 'W_Completeren aanvraag', 'W_Nabellen offertes', 'W_Nabellen offertes', 'W_Nabellen offertes', 'W_Nabellen offertes', 'W_Nabellen offertes', 'O_SENT_BACK', 'W_Valideren aanvraag', 'W_Nabellen offertes', 'W_Valideren aanvraag', 'O_ACCEPTED', 'A_REGISTERED', 'A_APPROVED', 'A_ACTIVATED', 'W_Valideren aanvraag']


<p> Calculate the distance between any given trace and all the other traces in the df. The distance to the other traces is put in another dictionary. Input is the trace we want to make a prediction for. Output is a dictionary with as keys the other case numbers and as values the distance to the given case. To test, we take a random case, 173688. Performance is surprisingly good. Uncomment the last line to see the calculated distances. </p> 

In [8]:
def calc_distances(event_dictionary, case_number):
    trace_distances = {}
    a = []
    a.clear()
    a.append(event_dictionary.get(case_number))

    for key, value in event_dictionary.items():
        if key == case_number:
            continue    
    
        for i in range (0, len(a)):
            d.append([i])

        for j in range (0, len(value)):
            d[0].insert(j, j)
    
        for i in range (1, len(a)):
            for j in range (1, len(value)):
                if a[i] == value[j]:
                    cost = 0
                else:
                    cost = 1
                    
                d[i].insert(j, min(d[i-1][j] + 1,
                             d[i][j-1] + 1,
                             d[i-1][j-1]+cost))
                if i>1 and j>1 and a[i] == value[j-1] and a[i-1] == value[j]:
                    d[i].insert(j, min(d[i][j], d[i-2][j-2] + 1))
                    d[i].pop(j+1)
                    
        trace_distances.update({key:d[len(a)-1][len(value)-1]})
        d.clear()
        
    a.clear()
    d.clear()
    return trace_distances

#calc_distances(case_events, 173688)
#calc_distances(case_events, 173691)

<p> Now we can use the dictionary trace_distances to select the traces we want to base predictions off of: either the x most similar cases or only cases with distances < x. This piece of code turns the dictionary into a useable dataframe (this can probably be done more efficiently but ok for now). To see either of the resulting dataframes uncomment one of the last lines.</p>

<p> These functions let you filter the dictionary either on max distance or the closest x cases. The second function is still a WIP. </p>

In [9]:
def filter_dictionary_maxdist(dictionary_old, max_dist):
    case_list = []
    for (key, value) in dictionary_old.items():
        if value <= max_dist:
            case_list.append(key)
    return case_list
    
def filter_dictionary_closestx(dictionary_old, n):
    case_list = sorted(dictionary_old, key=dictionary_old.get, reverse=True)[:n]
    return case_list

filter_dictionary_closestx(calc_distances(case_events, 173688), 10)
#filter_dictionary_maxdist(calc_distances(case_events, 173688), 10)

[185548,
 195247,
 183175,
 198232,
 181799,
 191797,
 188134,
 198253,
 183405,
 187217]

In [10]:
def filter_df(df, case_list):
    df_filtered = df[(df["case concept:name"].isin(case_list))]
    return df_filtered
    
filter_df(df_2012, filter_dictionary_maxdist(calc_distances(case_events, 173688), 10))

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
10,12884901888,173697,2011-10-01T08:11:08.865+02:00,15000,A_SUBMITTED,COMPLETE,01-10-2011 08:11:08.866
11,12884901889,173697,2011-10-01T08:11:08.865+02:00,15000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 08:11:09.035
14,12884901890,173697,2011-10-01T08:11:08.865+02:00,15000,A_DECLINED,COMPLETE,01-10-2011 08:11:46.420
15,17179869184,173700,2011-10-01T08:15:39.894+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:15:39.894
16,17179869185,173700,2011-10-01T08:15:39.894+02:00,5000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 08:15:40.102
17,17179869186,173700,2011-10-01T08:15:39.894+02:00,5000,A_DECLINED,COMPLETE,01-10-2011 08:16:21.037
18,21474836480,173703,2011-10-01T09:45:25.806+02:00,13500,A_SUBMITTED,COMPLETE,01-10-2011 09:45:25.806
19,21474836481,173703,2011-10-01T09:45:25.806+02:00,13500,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 09:45:25.981
22,21474836482,173703,2011-10-01T09:45:25.806+02:00,13500,A_PREACCEPTED,COMPLETE,01-10-2011 09:46:18.211
24,21474836483,173703,2011-10-01T09:45:25.806+02:00,13500,W_Completeren aanvraag,SCHEDULE,01-10-2011 09:46:18.674


<p> Putting the filtering of a dataframe together in a single function, we can filter a database with these calls: </p>
<p> (alternatively, there are also two functions that only return the list of similar traces) </p>

In [11]:
def filter_df_similar_maxdist(df_original, case_events, case_number, maxdist):
    distance_dict = calc_distances(case_events, case_number)
    distance_dict_filtered = filter_dictionary_maxdist(distance_dict, maxdist)
    df_filtered = filter_df(df_original, distance_dict_filtered)[["case concept:name", "event concept:name"]]
    return df_filtered

def filter_df_similar_closestx(df_original, case_events, case_number, x):
    distance_dict = calc_distances(case_events, case_number)
    distance_dict_filtered = filter_dictionary_closestx(distance_dict, x)
    df_filtered = filter_df(df_original, distance_dict_filtered)[["case concept:name", "event concept:name"]]
    return df_filtered

def list_similar_maxdist(case_events, case_number, maxdist):
    distance_dict = calc_distances(case_events, case_number)
    distance_dict_filtered = filter_dictionary_maxdist(distance_dict, maxdist)
    return distance_dict_filtered

def list_similar_closestx(case_events, case_number, x):
    distance_dict = calc_distances(case_events, case_number)
    distance_dict_filtered = filter_dictionary_closestx(distance_dict, x)
    return distance_dict_filtered
    
df_2012 = pd.read_csv('BPI_Challenge_2012-training.csv')
df_cols = df_filter_cols(df_2012)
case_events = create_case_dictionary(df_cols)

#df_end = filter_df_similar_maxdist(df_2012, case_events, 173688, 5)
df_end = filter_df_similar_closestx(df_2012, case_events, 173688, 5)
df_end

,case concept:name,event concept:name
42696,181799,A_SUBMITTED
42697,181799,A_PARTLYSUBMITTED
42699,181799,A_PREACCEPTED
42700,181799,W_Completeren aanvraag
42701,181799,W_Completeren aanvraag
42709,181799,W_Completeren aanvraag
44632,181799,W_Completeren aanvraag
44633,181799,W_Completeren aanvraag
45000,181799,W_Completeren aanvraag
45008,181799,W_Completeren aanvraag


<p> Create a new last_events dataframe that has all the case numbers and their last recorded activity </p>

In [12]:
df_cols.groupby(["case concept:name"], sort=False)["level_1"].max()
idx = df_cols.groupby(["case concept:name"], sort=False)["level_1"].transform(max) == df_cols['level_1']
df_last_events = df_cols[idx]
df_last_events.rename(columns={"case concept:name": "case_no", "event concept:name": "event_name"}, inplace=True)
df_last_events = df_last_events[["case_no", "event_name"]]
df_last_events

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,case_no,event_name
14,173697,A_DECLINED
17,173700,A_DECLINED
115,173727,A_DECLINED
143,173703,W_Completeren aanvraag
187,173733,W_Afhandelen leads
193,173706,W_Completeren aanvraag
196,173757,A_DECLINED
295,173769,A_DECLINED
305,173763,W_Afhandelen leads
310,173772,A_DECLINED


<p> Iterate over all the cases in df_last_events. In the inner loop, iterate over all of the recorded events and check if the same event occurs. If it does, add the next event to the eventlist (only if this event belongs to the same case) The ADDNEXT boolean variable was made to avoid having to get the row values of the next iteration. Instead, if I need to add the row value of the next iteration to the event list I pass a True ADDNEXT to the next iteration, after adding it to the list it is set back to false.</p>

In [ ]:
from collections import Counter
def get_first_mode(a):
    c = Counter(a)  
    mode_count = max(c.values())
    mode = {key for key, count in c.items() if count == mode_count}
    first_mode = next(x for x in a if x in mode)
    return first_mode

# def add_predicted_event(df_last_events, original_df, case_events, similarno)
for row in df_last_events.itertuples(index=True):
    df_filtered = filter_df_similar_closestx(df_2012, case_events, row[1], 20)
    mostcommonevent = 'bla'
    eventlist=['x']
    currentrow = row[0]
    currentcase = row[1]
    currentevent = row[2]
    ADDNEXT = False
    for row1 in df_filtered.itertuples(index=True):
        if ADDNEXT == True:
            eventlist.append(row1[2])
            ADDNEXT = False
        if row1[2] == currentevent:
            ADDNEXT = True
    mostcommonevent = get_first_mode(eventlist)
    df_last_events.loc[currentrow, "next_event"] = mostcommonevent
    eventlist.clear()
df_last_events